In [1]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer
import pandas as pd

c:\Users\DIEAL\.conda\envs\torchCuda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

#Separar el dataset en train y test (80% train, 20% test) Finaltrain.csv

df = pd.read_csv('data/Finaltrain.csv')
df = df.sample(frac=1).reset_index(drop=True)
train_data = df[:int(0.8*len(df))]
test_data = df[int(0.8*len(df)):]

train_data.to_csv('train.csv', index=False)
test_data.to_csv('test.csv', index=False)


In [3]:
train_data.head()

,student_id,prompt_id,text,content,wording,prompt_question,prompt_title,prompt_text,text_length,word_count,number_count,punctuation_count,stopword_count
0,4c4e75b3e4da,3b9047,The acient system of government for Egypt went...,0.531368,0.583991,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structure...,543,89,0,15,38
1,be955634f18d,3b9047,The structure of the ancient egyptian system o...,1.926590,0.668115,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structure...,981,173,0,13,82
2,9517face3a4e,39c16e,The three elements of an ideal tragedy is that...,0.018751,-1.294040,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to ...,406,70,0,24,34
3,bcb49ccaf79c,39c16e,the incidents and the plot are the end of the ...,-0.776512,-0.383759,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to ...,279,51,0,5,24
4,f8452ffa3291,ebad26,"Ways you can use,cover meat is like they chop...",-0.627647,-0.125597,Summarize the various ways the factory would u...,Excerpt from The Jungle,With one member trimming beef ...,283,56,0,5,30


In [4]:
train_data['text'].tolist()

["The acient system of government for Egypt went like this. Social calsses went from slaves, farmers/workers, traders, preits, and kings. This was involved in government because they each had to follow rules from the pharos and if they didn't they would have a punishment. The punishment could be Karma, or execution. Also they each had to pay the pharos taxes. ( From farmers) Priets were Also responsible for pleasing the gods and Pharos. These are reasons how government and how different social classes in government were involved during it.",
 'The structure of the ancient egyptian system of government was based highly on your status. It was similar to a pyramid. The lowest being slaves,peasants, and farmers. Once at the bottom it was very hard to get higher unless you worked extremely hard or many young boys whose parents saved money to send them to learn trades. The middle of the pyramid consisted of soldiers who looked over farmers and other in the lowers social class. Powerful noble

In [5]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer.batch_encode_plus(
    train_data['text'].tolist(),
    truncation=True,
    padding=True
)

test_encodings = tokenizer.batch_encode_plus(
    test_data['text'].tolist(),
    truncation=True,
    padding=True
)

train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(train_data['content'].tolist()),
    torch.tensor(train_data['wording'].tolist())
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask'])
)


In [6]:
class BERTModel(nn.Module):
  def __init__(self):
      super(BERTModel, self).__init__()
      self.bert = BertModel.from_pretrained('bert-base-uncased')

      self.dropout = nn.Dropout(0.1)
      self.linear1 = nn.Linear(768, 256)
      self.linear2 = nn.Linear(256, 2)

  def forward(self, input_ids, attention_mask):
      outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
      pooled_output = outputs.pooler_output
      pooled_output = self.dropout(pooled_output)
      output = self.linear1(pooled_output)
      output = nn.ReLU()(output)
      output = self.linear2(output)
      return output

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

model = BERTModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

cuda


In [8]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)

In [9]:
# Splitting training data into train and validation sets
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

# Creating validation loader
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)

In [ ]:
# Training loop
model.train()
for epoch in range(3):
    running_loss = 0.0
    cont = 0
    for step, (input_ids, attention_mask, content, wording) in enumerate(train_loader):
        print(cont)
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)
        content = content.to(device)
        wording = wording.to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs[:, 0], content) + criterion(outputs[:, 1], wording)
        loss.backward()
        optimizer.step()
        if step % 10 == 0:
            print("Epoch {}, Step {}, Loss: {}".format(epoch+1, step, loss.item()))

        running_loss += loss.item()

    print(f"Epoch {epoch+1} Loss: {running_loss / len(train_loader)}")

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0.0
        for val_step, (input_ids, attention_mask, content, wording) in enumerate(val_loader):
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            content = content.to(device)
            wording = wording.to(device)

            val_outputs = model(input_ids, attention_mask)
            val_loss += criterion(val_outputs[:, 0], content) + criterion(val_outputs[:, 1], wording)

        print(f"Validation Loss: {val_loss / len(val_loader)}")
    model.train()

c:\Users\DIEAL\.conda\envs\torchCuda\lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Epoch 1, Step 0, Loss: 1.8101425170898438


In [13]:
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

In [14]:
model.eval()
predictions = []
with torch.no_grad():
    for input_ids, attention_mask in test_loader:
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        outputs = model(input_ids, attention_mask)
        predictions.extend(outputs.cpu().numpy())

In [15]:
submission_df = pd.DataFrame({
    'student_id': test_data['student_id'],
    'content': [pred[0] for pred in predictions],
    'wording': [pred[1] for pred in predictions]
})

submission_df.to_csv('submission.csv', index=False)

In [16]:
submission_df

,student_id,content,wording
0,000000ffffff,-1.422887,-1.260434
1,111111eeeeee,-1.449072,-1.307865
2,222222cccccc,-1.464931,-1.377249
3,333333dddddd,-1.466634,-1.339841
